In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns 
from scipy.stats import binomtest,chisquare,ttest_1samp,chi2_contingency,ttest_ind,f_oneway,pearsonr
from sklearn.preprocessing import OneHotEncoder, LabelEncoder,MinMaxScaler,StandardScaler,PolynomialFeatures
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.feature_selection import SelectKBest,f_classif,f_regression
from sklearn.neighbors import LocalOutlierFactor


In [3]:
df = sns.load_dataset("titanic")
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# 11. La moyenne de l'âge (age) des survivants est-elle égale à celle des non-survivants ?

In [28]:
df.groupby("survived")['age'].mean()


survived
0    30.626179
1    28.343690
Name: age, dtype: float64

In [108]:
# Question 11 : Moyenne d'âge survivants = non-survivants ?
# H0 : μ_survivants = μ_non_survivants
# H1 : μ_survivants ≠ μ_non_survivants

alpha = 0.05

# Créer les 2 groupes (ARRAYS, pas moyennes)
age_unsurvived = df[df['survived'] == 0]['age'].dropna()
age_survived = df[df['survived'] == 1]['age'].dropna()

print(f"Moyenne âge non-survivants : {age_unsurvived.mean():.1f} ans")
print(f"Moyenne âge survivants : {age_survived.mean():.1f} ans")

# Test t indépendant
stat, p_value = ttest_ind(age_unsurvived, age_survived)

print(f"\nP-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ REJET H0")
    print("→ Les moyennes d'âge sont significativement DIFFÉRENTES entre survivants et non-survivants")
else:
    print("❌ ON NE REJETTE PAS H0")
    print("→ Pas de différence significative d'âge entre survivants et non-survivants")

Moyenne âge non-survivants : 30.6 ans
Moyenne âge survivants : 28.3 ans

P-value : 0.0391
✅ REJET H0
→ Les moyennes d'âge sont significativement DIFFÉRENTES entre survivants et non-survivants


# 12. La variable sex (genre) est-elle indépendante de la variable pclass (classe du  passager) ?

In [35]:
crosstab = pd.crosstab(df['sex'],df['pclass'])
alpha =0.02

P_value = chi2_contingency(crosstab).pvalue

if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")
    

nous n'avons pas assez dhypotheses pour rejetes H0


 # 13. Existe-t-il une corrélation linéaire entre le tarif payé (fare) et l'âge (age) du passager ?

In [109]:
# Question 13 : Corrélation linéaire entre fare et age ?
# H0 : Pas de corrélation (r = 0)
# H1 : Il existe une corrélation (r ≠ 0)

alpha = 0.05

# Enlever les NaN
data_clean = df[['fare', 'age']].dropna()

# Corrélation de Pearson
coef, p_value = pearsonr(data_clean['fare'], data_clean['age'])

print(f"Coefficient de corrélation : {coef:.3f}")
print(f"P-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ REJET H0")
    print(f"→ Il existe une corrélation linéaire significative entre fare et age")
    print(f"   (corrélation {'positive' if coef > 0 else 'négative'})")
else:
    print("❌ ON NE REJETTE PAS H0")
    print("→ Pas de corrélation linéaire significative entre fare et age")

Coefficient de corrélation : 0.096
P-value : 0.0102
✅ REJET H0
→ Il existe une corrélation linéaire significative entre fare et age
   (corrélation positive)


# 14. La proportion de femmes à bord est-elle significativement supérieure à 40% ?

In [113]:
# Question 14 : Proportion de femmes > 40% ?
# H0 : p ≤ 0.40 (40% ou moins)
# H1 : p > 0.40 (plus de 40%)
# ⚠️ TEST UNILATÉRAL DROITE

alpha = 0.05

k = (df['sex'] == 'female').sum()  # Nombre de femmes
n = len(df)  # Total passagers

proportion_observee = k / n

print(f"Proportion de femmes : {k}/{n} = {proportion_observee:.2%}")

# Test binomial UNILATÉRAL (greater)
result = binomtest(k=k, n=n, p=0.40, alternative='greater')
p_value = result.pvalue

print(f"P-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ REJET H0")
    print("→ La proportion de femmes est significativement SUPÉRIEURE à 40%")
else:
    print("❌ ON NE REJETTE PAS H0")
    print("→ La proportion de femmes n'est pas significativement supérieure à 40%")

Proportion de femmes : 314/891 = 35.24%
P-value : 0.9984
❌ ON NE REJETTE PAS H0
→ La proportion de femmes n'est pas significativement supérieure à 40%


In [116]:

k = df['sex'].value_counts()['female']
n = len(df)
p = 0.4
alpha = 0.02

p_value = binomtest(k=k,n=n,p=p,alternative='greater').pvalue

if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")


nous n'avons pas assez dhypotheses pour rejetes H0


# 15. Le tarif moyen (fare) est-il significativement différent entre les trois classes de  passagers (pclass) ?

In [76]:
group1 = df[df['pclass'] == 1]['fare']
group2 = df[df['pclass'] == 2]['fare']
group3 = df[df['pclass'] == 3]['fare']


alpha = 0.02
p_value = f_oneway(group1,group2,group3).pvalue

if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")

nous avons  assez dhypotheses pour rejeter H0


# 16. La moyenne d'âge des passagers est-elle inférieure à 35 ans ?

In [77]:
df['age'].describe()

count    714.000000
mean      29.699118
std       14.526497
min        0.420000
25%       20.125000
50%       28.000000
75%       38.000000
max       80.000000
Name: age, dtype: float64

In [79]:
alpha = 0.02
p_value = ttest_1samp(df['age'],popmean=35,alternative='less').pvalue
if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")

nous n'avons pas assez dhypotheses pour rejetes H0


# 17. Y a-t-il une relation entre le port d'embarquement (embarked) et le fait d'avoir survécu (survived) ?

In [81]:
crosstab2 = pd.crosstab(df['embarked'],df['survived'])
alpha = 0.02

p_value = chi2_contingency(crosstab2).pvalue
if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")

nous avons  assez dhypotheses pour rejeter H0


# 8. La moyenne du tarif payé (fare) est-elle différente pour les passagers de la Première Classe et ceux de la Troisième Classe ?


In [84]:
groupe_1 = df[df['pclass']==1]
group2_3 = df[df['pclass'] ==3]
alpha = 0.02

p_value = ttest_ind(groupe_1['fare'],group2_3['fare']).pvalue

if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")

nous avons  assez dhypotheses pour rejeter H0


# 19. La distribution des passagers entre les trois ports d'embarquement est-elle uniforme ?

In [110]:
# Question 19 : Distribution uniforme entre les 3 ports ?
# H0 : Distribution UNIFORME (1/3, 1/3, 1/3)
# H1 : Distribution NON UNIFORME

alpha = 0.05

# Fréquences observées
observed = df['embarked'].value_counts().sort_index().values
n_total = observed.sum()
n_categories = len(observed)

print(f"Fréquences observées : {observed}")
print(f"Ports : {df['embarked'].value_counts().sort_index().index.tolist()}")

# Fréquences attendues (uniformes)
expected = np.array([n_total / n_categories] * n_categories)
print(f"Fréquences attendues (uniforme) : {expected}")

# Test du Chi² d'ajustement
chi2, p_value = chisquare(f_obs=observed, f_exp=expected)

print(f"\nChi² = {chi2:.2f}")
print(f"P-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ REJET H0")
    print("→ La distribution des passagers entre les ports N'EST PAS uniforme")
else:
    print("❌ ON NE REJETTE PAS H0")
    print("→ La distribution des passagers peut être considérée comme uniforme")

Fréquences observées : [168  77 644]
Ports : ['C', 'Q', 'S']
Fréquences attendues (uniforme) : [296.33333333 296.33333333 296.33333333]

Chi² = 625.81
P-value : 0.0000
✅ REJET H0
→ La distribution des passagers entre les ports N'EST PAS uniforme


# 20. Y a-t-il une relation linéaire entre le montant du tarif payé (fare) et le fait d'avoir survécu (survived) ?

In [107]:
alpha = 0.02

p_value = ttest_ind(df['fare'],df['survived']).pvalue
if p_value < alpha :
    print("nous avons  assez dhypotheses pour rejeter H0")
else:
    print("nous n'avons pas assez dhypotheses pour rejetes H0")


nous avons  assez dhypotheses pour rejeter H0


In [111]:
# Question 20 : Alternative - Comparer fare moyen entre survivants et non-survivants

fare_non_survivants = df[df['survived'] == 0]['fare'].dropna()
fare_survivants = df[df['survived'] == 1]['fare'].dropna()

print(f"Fare moyen non-survivants : {fare_non_survivants.mean():.2f}$")
print(f"Fare moyen survivants : {fare_survivants.mean():.2f}$")

stat, p_value = ttest_ind(fare_non_survivants, fare_survivants)

print(f"\nP-value : {p_value:.4f}")

if p_value < alpha:
    print("✅ REJET H0")
    print("→ Le tarif moyen est différent entre survivants et non-survivants")
else:
    print("❌ ON NE REJETTE PAS H0")
    print("→ Pas de différence de tarif entre survivants et non-survivants")

Fare moyen non-survivants : 22.12$
Fare moyen survivants : 48.40$

P-value : 0.0000
✅ REJET H0
→ Le tarif moyen est différent entre survivants et non-survivants
